In [3]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import json
import copy

In [4]:
#NOTE: THERE IS A PROBLEM WITH THE STATS, THEY DON'T SEEM UP TO DATE
#NEED TO RETHINK THE SCRAPING METHOD


st = pd.read_pickle('../IGNOBEL/Statistiche_giocatori.pkl') #importing complete stats DB
transStat = copy.deepcopy(st)
transStat.index=list(st.Id)
transStat=transStat.T # transposed df makes it easier to access players by id
#print(transStat.index)
transStat

,2610,2531,785,441,2530,531,507,2819,1874,2764,...,2773,4346,5013,5297,5303,2013,4387,5322,4335,4938
Id,2610,2531,785,441,2530,531,507,2819,1874,2764,...,2773,4346,5013,5297,5303,2013,4387,5322,4335,4938
R,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
Nome,RONALDO,LUKAKU,IMMOBILE,BELOTTI,IBRAHIMOVIC,BERARDI,MURIEL,CAPUTO,JOAO PEDRO,MARTINEZ L.,...,SPROCATI,MILLICO,ASORO,LUVUMBO,MALES,RICCI F.,ADORANTE,MIHAILA,SCHIAPPACASSE,MORO
Squadra,Juventus,Inter,Lazio,Torino,Milan,Sassuolo,Atalanta,Sassuolo,Cagliari,Inter,...,Parma,Torino,Genoa,Cagliari,Genoa,Sassuolo,Parma,Parma,Sassuolo,Lazio
Pg,10,13,12,13,6,13,11,9,14,14,...,1,3,0,0,0,1,1,0,1,0
Mv,6.5,6.71,6.46,6.85,7.25,6.65,6.72,6.44,6.36,6.46,...,5.5,6,0,0,0,0,0,0,6,0
Mf,9.8,9.46,8.46,9.04,11.25,8.42,9.28,8.72,8.18,7.89,...,5.5,6,0,0,0,0,0,0,6,0
Gf,8,8,7,7,8,5,6,5,6,6,...,0,0,0,0,0,0,0,0,0,0
Gs,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Rp,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# DB - DICTIONARY SKELETON # 

# Dictionary with specific loan info
LoanDict = {
    'start_date': '01/08/2020',
    'expire_date': '31/07/2021',
    'cost': 0,
    'redemption_right': False, #True or False
    'redemption_cost': 0, #>0 if redemption_right=True
}

# Dictionary with player current team and loan info
CurrentTeamDict = {
    'owner': '', #luca, pietro etc
    #'team': '', #team_name 
    'squad': '', # 'Main' or 'Primavera'
    'start_date': '01/08/2020',
    'previous_team': '', #last team (e.g. owner, squad)
    'quotation_initial': 0,
    'on_loan': False, #True or False
    'loan_info': None
}

# Dictionary with player ownership info
ContractDict = {
    'owner': '', #this seems redundant
    'start_date': '01/08/2020',
    'cost': 0,
    'acquisition_mode': '', #asta_svincolati, draft, acquisto
    'previous_owner': '', #owner or None
    'quotation_initial': 0
}

# Dictionary with player personal info
PersonalInfoDict = {
    'full_name': '',
    'birthdate': '01/01/1970',
    'nation': '',
    'team_real': '',
    'FC_role': ''
}

# Nested dictionary for a single player info
PlayerDict = {
    'id': 0,
    'name': '',
    'personal_info': PersonalInfoDict,
    'contract': ContractDict,
    'current_team': CurrentTeamDict,
}

# Main nested dictionary with ALL FPF players info
PlayersDB = {}

In [4]:
# IMPORT CURRENT DATA INTO DB DICTIONARY #
for Id, Name in zip(st['Id'], st['Nome']):
    PlayersDB[Id] = copy.deepcopy(PlayerDict)
    PlayersDB[Id]['id'] = Id
    PlayersDB[Id]['name'] = Name

#print(PlayersDB)

with open('FPF_Players_Database.json', 'w') as fp:
    json.dump(PlayersDB, fp)

# Updating entire database with stats and info

Do not run this section unless something is changed in the stats database

In [5]:
def check_prim(text):
    if ' - PRIMAVERA' in text:
        out = text[:-12]
        squad = 'primavera'
    else:
        out = text
        squad = 'main'
    return out, squad

In [6]:
PlayersDB[2610]

{'id': 2610,
 'name': 'RONALDO',
 'personal_info': {'full_name': '',
  'birthdate': '01/01/1970',
  'nation': '',
  'team_real': '',
  'FC_role': ''},
 'contract': {'owner': '',
  'start_date': '01/08/2020',
  'cost': 0,
  'acquisition_mode': '',
  'previous_owner': '',
  'quotation_initial': 0},
 'current_team': {'owner': '',
  'squad': '',
  'start_date': '01/08/2020',
  'previous_team': '',
  'quotation_initial': 0,
  'on_loan': False,
  'loan_info': None}}

In [21]:
for Id, dic in PlayersDB.items():
    dic['personal_info']['FC_role'] = transStat[Id]['R']
    dic['personal_info']['team_real'] = transStat[Id]['Squadra']
    dic['personal_info']['full_name'] = transStat[Id]['Nome Completo']
    dic['personal_info']['nation'] = transStat[Id]['Nazionalit\'a']
    dic['personal_info']['birthdate'] = transStat[Id]['Classe']
    #dic['personal_info']['age'] = transStat[Id]['Eta\'']
    dic['contract']['quotation_initial'] = transStat[Id]['Qt. I']
    dic['contract']['owner'] = transStat[Id]['Allenatore']
    dic['current_team']['owner'] = transStat[Id]['Allenatore']
    pl_hold, dic['current_team']['squad']  = check_prim(transStat[Id]['Nome Squadra'])
    dic['current_team']['quotation_initial'] = transStat[Id]['Qt. I'] 
    temp = dict(transStat[Id][4:20])
    temp['Qt_I'] = temp.pop('Qt. I')
    temp['Qt_A'] = temp.pop('Qt. A')
    dic['stats'] = temp
    
# COMMENTED TO PREVENT OVERWRITING    
with open('FPF_Players_Database.json', 'w') as fp:
    json.dump(PlayersDB, fp)

In [22]:
# WRITE PLAYER IDS TO TXT FILE #
f = open("PlayersIds.txt", "w")
f.write('Id Name\n')
for Id,Name in zip(st['Id'], st['Nome']):
    f.write(str(Id)+' '+Name+'\n')
f.close()

# Import created database and upload it to mongoDB

## Note: I need to fix the credentials. For now I have to add the ip address of the machine connecting to the db, so it is not ideal because I'm not using static ip 

In [23]:
from pymongo import MongoClient
from pprint import pprint
import pymongo

with open('FPF_Players_Database.json','r') as f:
    PlayersDB_in = json.load(f)


with open('credential.json','r') as f:
    cred = json.load(f)

In [24]:
#Creating a list of dictionaries, which is the preferred structure for MongoDB
posts = []
for idx, dic in PlayersDB_in.items():
    #Note: the change to label '_id' is to respect MongoDB standards
    #temp = {"_id": dic['id'], "name": dic['name'],"owner": dic['current_team']['owner'],"team": dic['current_team']['team'],"squad": dic['current_team']['squad'],"real_team": dic['personal_info']['team_real'],"role": dic['personal_info']['FC_role'],"info": dic}
    temp = {"_id": dic['id'], "name": dic['name'], "info": dic}
    temp['info'].pop('id')
    temp['info'].pop('name')
                                         
    posts.append(temp)

In [25]:
#Inserting database for the first time
# specifying cluster with credential
with open('credential.json','r') as f:
    PlayersDB_in = json.load(f)

    
cluster = MongoClient(cred['cred'])
# choosing database
db = cluster["Game"]
# choosing collection
collection = db["Players"]

collection.delete_many({})
collection.insert_many(posts)

In [29]:
dict_names={
    'enzo': 'AS 800A' ,
    'pietro':'PDG 1908' ,
    'mario':'IGNORANZA EVERYWHERE' ,
    'musci8':'SOROS FC' ,
    'franky':'MAINZ NA GIOIA' ,
    'nanni':'PALLA PAZZA' ,
    'emiliano':'I DISEREDATI',
    'luca':'XYZ' 
    }

posts_Managers = []

i=1
for name, team in dict_names.items():
    dic_man = {
        '_id': i,
        'owner': name,
        'team_name': team,
        'budget': 350,
        'palmares': [], #this will be an array of dictionaries, each containing info on the trophies
        'fines':[], #this is an array of dictionaries, each with info on the specific fine
        'total_wins': 0
    }
    posts_Managers.append(dic_man)
    i = i+1
collectionM = db["Managers"]
collectionM.delete_many({})
collectionM.insert_many(posts_Managers)

### Every entry in the database can be queried by external keys

Note: mongoDB allows you to query (either to find, delete or replace) by the keys in the outmost dictionary, not the nested dictionary, so we need to decide what we want to have as queriable keys and flatten those.

We can query by:

- id (key '_id') this follows mongoDB standards and it MUST BE unique
- name
- owner
- team
- squad
- real_team
- role

We can think whether to add more queriable keys, for instance:

- birthdate? (maybe just year)

In MongoDB you can easily cross these queries and, for instance, find all the players that match multiple criteria (examples below)

In [15]:
#Example of a find query by name: 
#find the element that matches the criteria
#Note: it makes sense if you search unique elements, 
#if you search by team it gives only first result, in that case see below
result = collection.find_one({"name":'MUSSO'})

pprint(result) #better way to print dictionaries

{'_id': 2792,
 'info': {'contract': {'acquisition_mode': '',
                       'cost': 0,
                       'owner': 'pietro',
                       'previous_owner': '',
                       'quotation_initial': 11,
                       'start_date': '01/08/2020'},
          'current_team': {'on_loan': False,
                           'owner': 'pietro',
                           'previous_team': '',
                           'quotation_initial': 11,
                           'squad': 'main',
                           'start_date': '01/08/2020'},
          'personal_info': {'FC_role': 'P',
                            'birthdate': '06/05/1994',
                            'full_name': 'Musso Juan',
                            'nation': 'Argentina, Italia',
                            'team_real': 'Udinese'},
          'stats': {'Amm': 0,
                    'Asf': 0,
                    'Ass': 0,
                    'Au': 0,
                    'Esp': 0,
            

In [16]:
#Example, single criteria with multiple matches
#Find all Atalanta players

results = collection.find({'info.personal_info.team_real': 'Atalanta'})
for result in results:
    print(result['name']) #printing just names for convenience

ZAPATA D.
GOMEZ
MURIEL
GOSENS
ILICIC
MALINOVSKYI
PASALIC
MIRANCHUK
HATEBOER
LAMMERS
GOLLINI
TOLOI
FREULER
DE ROON
PALOMINO
DJIMSITI
ROMERO
PICCINI
PESSINA
CALDARA
DEPAOLI
MOJICA
SPORTIELLO
SUTALO
CARNESECCHI
ROSSI F.
RADUNOVIC
DIALLO


In [157]:
#Example of multiple criteria: 
#Find all Lazio players in pietro's team

results = collection.find({'info.contract.owner': 'pietro', 'info.personal_info.team_real': 'Bologna'}) #query order is irrelevant

#it returns a cursor, so you need to loop over it to see the content
for result in results:
    print(result['name']) #printing just names for convenience

SANSONE


In [159]:
#Example of multiple criteria: 
#Find all main squad players in pietro's team

results = collection.find({'info.current_team.owner': 'pietro', 'info.current_team.squad': 'main'})
for result in results:
    print(result['name']) #printing just names for convenience

ZAPATA D.
MURIEL
LUIS ALBERTO
CALHANOGLU
CHIESA
HERNANDEZ T.
DE PAUL
RAFAEL LEAO
POLITANO
LOZANO
GABBIADINI
MANOLAS
BONAZZOLI
LAZZARI
SPINAZZOLA
SENSI
DRAGOWSKI
MUSSO
D'AMBROSIO
CANDREVA
GLIK
SKRINIAR
TOLJAN
MONTIPO'
PELLEGRINI LU.


In [160]:
#Example of multiple criteria: 
#Find all primavera squad players in luca's team who play for Verona

results = collection.find({'info.current_team.owner': 'luca', 'info.current_team.squad': 'primavera', 'info.personal_info.team_real': 'Verona'})
for result in results:
    print(result['name']) #printing just names for convenience

FAVILLI
SALCEDO E.


### Deleting elements from the database

I left every cell commented to avoid erasing elements by mistake

In [65]:
#deleting a single element that matches the criteria: (DON'T RUN)

#THIS deletes the entire entry that matches the chosen criteria (name)
# results = collection.delete_one({"name": "MUSSO"})

In [ ]:
# THIS deletes every entry that matches this criteria (we don't have duplicates, so it is always one)
#
# results = collection.delete_many({"name": "MUSSO"})

In [118]:
# USE THIS ONLY IF YOU WANT TO DELETE ALL:

#results = collection.delete_many({})

### Update elements in the database

The general command is the following 

```collection.update_many(<criteria>, {$set:{<key>: <new value>}})
```
    
Since we have a nested dictionary and we need to modify multiple entries at the same time (e.g. owner and team ) the best logic to follow is: 
- to download the entire entry using find and assign it to a dictionary
- modify the entire dictionary
- update the entire element

There are several ways of doing it but we definitely need 2 minimal components:
- One function that given the dictionary "info" returns an updated element in the form of the content of the array 'posts' defined above
- one function that downloads the info database (using whatever query you want), calls the previous function to create the new element

In any case, we need to make the functions as modular as possible, so that we need do define only a few

### Access to nested keys via dot notation 
We can update any element of the nested disctionaries

In [165]:
# Example, Updating a nested key 
collection.update_one({'name': 'DANILO LAR.'}, {'$set':{'info.contract.acquisition_mode': 'Scambio'}})
result = collection.find_one({"name":'BELOTTI'})
pprint(result) #better way to print dictionaries

{'_id': 441,
 'info': {'contract': {'acquisition_mode': '',
                       'cost': 0,
                       'owner': 'luca',
                       'previous_owner': '',
                       'quotation_initial': 29,
                       'start_date': '01/08/2020'},
          'current_team': {'on_loan': False,
                           'owner': 'luca',
                           'previous_team': '',
                           'quotation_initial': 29,
                           'squad': 'main',
                           'start_date': '01/08/2020',
                           'team': 'XYZ'},
          'personal_info': {'FC_role': 'A',
                            'age': '27',
                            'birthdate': '20/12/1993',
                            'full_name': 'Belotti Andrea',
                            'nation': 'Italia',
                            'team_real': 'Torino'},
          'stats': {'Amm': 0,
                    'Asf': 0,
                    'Ass': 1,
 

In [166]:
#Example, find elements by nested key query

results = collection.find({'info.current_team.quotation_initial': 11})
for result in results:
    print(result['name']) #printing just names for convenience

LOZANO
CAPRARI
SILVESTRI
HATEBOER
LAZZARI
JANKTO
DEFREL
LAMMERS
CHIELLINI
FREULER
BARAK
PEREYRA
CRAGNO
MUSSO
N'KOULOU
SMALLING
ZAJC
PINAMONTI
SIRIGU
DI CARMINE


# In the following section we define the functions that we need to manipulate the database